In [3]:
import numpy as np
import pandas as pd
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.linear_model import LogisticRegression
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
# Replace the deprecated function with the recommended one
import tensorflow as tf

# Old function
# from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
import time
import os

import matplotlib.pyplot as plt 
import seaborn as sns


In [66]:
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from wordcloud import WordCloud,STOPWORDS
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
print(os.listdir(r"C:\Users\sumat\OneDrive\Documents\IND_HR"))

['ARUNACHAL_PRADESH-2011.csv', 'ASSAM-2011.csv', 'BIHAR-2011.csv', 'DELHI-2011.csv', 'df_final.csv', 'GOA-2011.csv', 'GUJARAT-2011.csv', 'HIMACHAL_PRADESH-2011.csv', 'JHARKHAND-2011.csv', 'KARNATAKA-2011.csv', 'KERALA-2011.csv', 'MAHARASHTRA-2011.csv', 'MANIPUR-2011.csv', 'merged_df_filtered.csv', 'merged_file.csv', 'MIZORAM-2011.csv', 'NAGALAND-2011.csv', 'ODISHA-2011.csv', 'PUDUCHERRY-2011.csv', 'RAJASTHAN-2011.csv', 'SIKKIM-2011.csv', 'TAMIL_NADU-2011.csv', 'TRIPURA-2011.csv', 'UTTARAKHAND-2011.csv', 'UTTAR_PRADESH-2011.csv', 'WEST_BENGAL-2011.csv']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sumat\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


Table of Contents: Dataset Preparation: The first step is the Dataset Preparation step which includes the process of loading a dataset and performing basic pre-processing. The dataset is then splitted into train and validation sets. Feature Engineering: The next step is the Feature Engineering in which the raw dataset is transformed into flat features which can be used in a machine learning model. This step also includes the process of creating new features from the existing data. Model Training: The final step is the Model Building step in which a machine learning model is trained on a labelled dataset. Improve Performance of Text Classifier: In this article, we will also look at the different ways to improve the performance of text classifiers.

# Loading the Training Data

In [114]:
train = pd.read_csv(r"C:\Users\sumat\OneDrive\Documents\IND_HR\df_final.csv")
train.head(2)

,states,districts,division,group,class,nic_name,main_workers_rural_males,main_workers_rural_females,main_workers_urban_males,main_workers_urban_females,marginal_workers_rural_males,marginal_workers_rural_females,marginal_workers_urban_males,marginal_workers_urban_females,category
0,ARUNACHAL PRADESH,District - Tawang,`01,`000,`0000,"Crop and animal production, hunting and relate...",300,84,4,5,46,62,2,0,"Crop and animal production, hunting and relate..."
1,ARUNACHAL PRADESH,District - Tawang,`01,`011,`0000,Growing of non-perennial crops,16,8,0,1,14,16,2,0,"Crop and animal production, hunting and relate..."


In [135]:
#Identifying the first Tag for all rows
first_tag = []
for str in train['nic_name']:
    first_tag.append(str.split(';')[0])

In [136]:
#Number of Records in First_Tag
len(first_tag)

176609

In [137]:
#Adding a new feature into the training dataset
train['First_Tag'] = first_tag
train.head(2)

,states,districts,division,group,class,nic_name,main_workers_rural_males,main_workers_rural_females,main_workers_urban_males,main_workers_urban_females,marginal_workers_rural_males,marginal_workers_rural_females,marginal_workers_urban_males,marginal_workers_urban_females,category,First_Tag
0,ARUNACHAL PRADESH,District - Tawang,`01,`000,`0000,"Crop and animal production, hunting and relate...",300,84,4,5,46,62,2,0,"Crop and animal production, hunting and relate...","Crop and animal production, hunting and relate..."
1,ARUNACHAL PRADESH,District - Tawang,`01,`011,`0000,Growing of non-perennial crops,16,8,0,1,14,16,2,0,"Crop and animal production, hunting and relate...",Growing of non-perennial crops


# Unique Value Count for each feature

In [138]:
#Unique Values for Geographic Locations
train['states'].unique()

array(['ARUNACHAL PRADESH', 'ASSAM', 'Bihar', 'NCT OF DELHI', 'GOA',
       'Gujarat', 'HIMACHAL PRADESH', 'JHARKHAND', 'KARNATAKA', 'KERALA',
       'MAHARASHTRA', 'MANIPUR', 'NAGALAND', 'ODISHA', 'PUDUCHERRY',
       'RAJASTHAN', 'SIKKIM', 'TAMIL NADU', 'TRIPURA', 'UTTARAKHAND',
       'UTTAR PRADESH', 'WEST BENGAL'], dtype=object)

In [139]:
train['districts'].unique()

array(['District - Tawang', 'District - West Kameng',
       'District - East Kameng', 'District - Papum Pare',
       'District - Upper Subansiri', 'District - West Siang',
       'District - East Siang', 'District - Upper Siang',
       'District - Changlang', 'District - Tirap',
       'District - Lower Subansiri', 'District - Kurung Kumey',
       'District - Dibang Valley', 'District - Lower Dibang Valley',
       'District - Lohit', 'District - Anjaw', 'District - Kokrajhar',
       'District - Dhubri', 'District - Goalpara', 'District - Barpeta',
       'District - Morigaon', 'District - Nagaon', 'District - Sonitpur',
       'District - Lakhimpur', 'District - Dhemaji',
       'District - Tinsukia', 'District - Dibrugarh',
       'District - Sivasagar', 'District - Jorhat', 'District - Golaghat',
       'District - Karbi Anglong', 'District - Dima Hasao',
       'District - Cachar', 'District - Karimganj',
       'District - Hailakandi', 'District - Bongaigaon',
       'Distric

In [140]:
#Unique Values for CompanyType
train['division'].unique()

array(['`01', '`02', '`03', '`05', '`08', '`09', '`10', '`13', '`14',
       '`15', '`16', '`18', '`20', '`23', '`24', '`25', '`31', '`32',
       '`33', '`35', '`36', '`41', '`42', '`43', '`45', '`46', '`47',
       '`49', '`51', '`52', '`53', '`55', '`56', '`60', '`61', '`62',
       '`63', '`64', '`65', '`68', '`69', '`75', '`77', '`79', '`80',
       '`81', '`82', '`84', '`85', '`86', '`88', '`90', '`91', '`92',
       '`93', '`94', '`95', '`96', '`97', '`99', '`11', '`17', '`22',
       '`26', '`28', '`30', '`37', '`38', '`58', '`59', '`66', '`74',
       '`87', '`98', '`06', '`19', '`21', '`71', '`72', '`39', '`12',
       '`50', '`27', '`07', '`73', '`78', '`70', '`29'], dtype=object)

In [141]:
#Abstracting States,NIC Name,Category from train into a new dataset
train_ana = train[['states','nic_name','First_Tag']]
train_ana.head(2)

,states,nic_name,First_Tag
0,ARUNACHAL PRADESH,"Crop and animal production, hunting and relate...","Crop and animal production, hunting and relate..."
1,ARUNACHAL PRADESH,Growing of non-perennial crops,Growing of non-perennial crops


We will now carry out all the Text Preprocessing steps on the 'nic_name' field converting all letters to lower or upper case converting numbers into words or removing numbers removing punctuations, accent marks and other diacritics removing white spaces expanding abbreviations removing stop words, sparse terms, and particular words text canonicalization

# Convert text to lowercase

In [142]:
import pandas as pd

train_ana['Tidy_Desc'] = train_ana['nic_name'].str.lower()

train_ana.head(1)


C:\Users\sumat\AppData\Local\Temp\ipykernel_10016\3428481223.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ana['Tidy_Desc'] = train_ana['nic_name'].str.lower()


,states,nic_name,First_Tag,Tidy_Desc
0,ARUNACHAL PRADESH,"Crop and animal production, hunting and relate...","Crop and animal production, hunting and relate...","crop and animal production, hunting and relate..."


In [143]:
import re
#Function to remove any additional special characters, if needed.
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [125]:
#Facts: Translate function has been changed from Python 2.x to Python 3.x
#It now takes only one output
pattern = r'\d'  # Remove all digits from the text
train_ana['Tidy_Desc'] = train_ana['Tidy_Desc'].apply(lambda x: remove_pattern(x, pattern))

C:\Users\sumat\AppData\Local\Temp\ipykernel_10016\104769970.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ana['Tidy_Desc'] = train_ana['Tidy_Desc'].apply(lambda x: remove_pattern(x, pattern))


In [144]:
train_ana.head(1)

,states,nic_name,First_Tag,Tidy_Desc
0,ARUNACHAL PRADESH,"Crop and animal production, hunting and relate...","Crop and animal production, hunting and relate...","crop and animal production, hunting and relate..."


# Removing punctuations, accent marks and other diacritics

In [145]:
#[!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]
train_ana.Tidy_Desc=train_ana.Tidy_Desc.apply(lambda x: x.translate({ord(c):'' for c in "[!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]"}))

C:\Users\sumat\AppData\Local\Temp\ipykernel_10016\3890938482.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ana.Tidy_Desc=train_ana.Tidy_Desc.apply(lambda x: x.translate({ord(c):'' for c in "[!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]"}))


In [146]:
train_ana.head(1)

,states,nic_name,First_Tag,Tidy_Desc
0,ARUNACHAL PRADESH,"Crop and animal production, hunting and relate...","Crop and animal production, hunting and relate...",crop and animal production hunting and related...


# Remove whitespaces

In [147]:
train_ana.Tidy_Desc=train_ana.Tidy_Desc.apply(lambda x: x.strip())

C:\Users\sumat\AppData\Local\Temp\ipykernel_10016\2728746437.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ana.Tidy_Desc=train_ana.Tidy_Desc.apply(lambda x: x.strip())


In [148]:
train_ana.head(1)

,states,nic_name,First_Tag,Tidy_Desc
0,ARUNACHAL PRADESH,"Crop and animal production, hunting and relate...","Crop and animal production, hunting and relate...",crop and animal production hunting and related...


Tokenization
Tokenization is the process of splitting the given text into smaller pieces called tokens.

# StopWord Removal + Tokenization

In [149]:
stop_words = stopwords.words('english')
from nltk.tokenize import word_tokenize

In [150]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [151]:
clean_sentences  = [remove_stopwords(r.split()) for r in train_ana['Tidy_Desc']]

In [152]:
train_ana['Tidy_Desc'] = clean_sentences

C:\Users\sumat\AppData\Local\Temp\ipykernel_10016\2101395835.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ana['Tidy_Desc'] = clean_sentences


In [153]:
train_ana.head(3)

,states,nic_name,First_Tag,Tidy_Desc
0,ARUNACHAL PRADESH,"Crop and animal production, hunting and relate...","Crop and animal production, hunting and relate...",crop animal production hunting related service...
1,ARUNACHAL PRADESH,Growing of non-perennial crops,Growing of non-perennial crops,growing nonperennial crops
2,ARUNACHAL PRADESH,Growing of rice,Growing of rice,growing rice


In [72]:
#Tokenization
tokens = train_ana['Tidy_Desc'].apply(lambda x: x.split())
#Now that we have the removed StopWords and tokenized the nic_name
#We will now subject the tokenized version to removing stop words, sparse terms, and particular words
#In some cases, it’s necessary to remove sparse terms or particular words from texts. 
#This task can be done using stop words removal techniques considering that any group of words can be chosen as the stop words.

In [154]:
#Stemming
from nltk.stem import PorterStemmer
stemmer= PorterStemmer()
Stemmed_tokens = tokens.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming

Now let’s stitch these tokens back together.

In [155]:
for i in range(len(Stemmed_tokens)):
    Stemmed_tokens[i] = ' '.join(Stemmed_tokens[i])

train_ana['Tidy_Desc_Stemmed'] = Stemmed_tokens

C:\Users\sumat\AppData\Local\Temp\ipykernel_10016\3784480293.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ana['Tidy_Desc_Stemmed'] = Stemmed_tokens


In [156]:
train_ana.head(3)

,states,nic_name,First_Tag,Tidy_Desc,Tidy_Desc_Stemmed
0,ARUNACHAL PRADESH,"Crop and animal production, hunting and relate...","Crop and animal production, hunting and relate...",crop animal production hunting related service...,crop anim product hunt relat servic activ
1,ARUNACHAL PRADESH,Growing of non-perennial crops,Growing of non-perennial crops,growing nonperennial crops,grow nonperenni crop
2,ARUNACHAL PRADESH,Growing of rice,Growing of rice,growing rice,grow rice


In [157]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sumat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [158]:
#Lemmatization
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
Lemmatized_tokens = tokens.apply(lambda x: [lemmatizer.lemmatize(i) for i in x]) # Lemmatizing

In [159]:
for i in range(len(Lemmatized_tokens)):
    Lemmatized_tokens[i] = ' '.join(Lemmatized_tokens[i])

train_ana['Tidy_Desc_Lemma'] = Lemmatized_tokens

C:\Users\sumat\AppData\Local\Temp\ipykernel_10016\1327823349.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ana['Tidy_Desc_Lemma'] = Lemmatized_tokens


In [160]:
train_ana.head(3)

,states,nic_name,First_Tag,Tidy_Desc,Tidy_Desc_Stemmed,Tidy_Desc_Lemma
0,ARUNACHAL PRADESH,"Crop and animal production, hunting and relate...","Crop and animal production, hunting and relate...",crop animal production hunting related service...,crop anim product hunt relat servic activ,crop animal production hunting related service...
1,ARUNACHAL PRADESH,Growing of non-perennial crops,Growing of non-perennial crops,growing nonperennial crops,grow nonperenni crop,growing nonperennial crop
2,ARUNACHAL PRADESH,Growing of rice,Growing of rice,growing rice,grow rice,growing rice


In [161]:
#Before going for Count Vectorization as Feature
#We would like to check the type of Count of Type of Tags

train_ana['First_Tag'].value_counts().head(5)

Construction of buildings                                      1353
Activities of households as employers of domestic personnel    1218
Manufacture of furniture                                       1146
Water collection, treatment and supply                         1104
Veterinary activities                                          1029
Name: First_Tag, dtype: int64

Count Vectors as features

In [162]:
#set(train_ana['First_Tag'])
#Selecting the first 10 labels
T10Tag = train_ana['First_Tag'].value_counts().index.tolist()
T10Tag = T10Tag[:10]
T10Tag

['Construction of buildings',
 'Activities of households as employers of domestic personnel',
 'Manufacture of furniture',
 'Water collection, treatment and supply',
 'Veterinary activities',
 'Incomplete description/ Wrongly Classifed',
 'Primary education',
 'Blank',
 'Manufacture of tobacco products',
 'Other human health activities']

In [163]:
train_10 = train_ana[train_ana['First_Tag'].isin(T10Tag)]
train_10.shape

(10202, 6)

In [164]:
set(train_10['First_Tag'])

{'Activities of households as employers of domestic personnel',
 'Blank',
 'Construction of buildings',
 'Incomplete description/ Wrongly Classifed',
 'Manufacture of furniture',
 'Manufacture of tobacco products',
 'Other human health activities',
 'Primary education',
 'Veterinary activities',
 'Water collection, treatment and supply'}

In [165]:
from collections import Counter
Counter(train_10['First_Tag'])

Counter({'Manufacture of furniture': 1146,
         'Water collection, treatment and supply': 1104,
         'Construction of buildings': 1353,
         'Veterinary activities': 1029,
         'Primary education': 880,
         'Other human health activities': 850,
         'Activities of households as employers of domestic personnel': 1218,
         'Blank': 860,
         'Incomplete description/ Wrongly Classifed': 904,
         'Manufacture of tobacco products': 858})

In [166]:
train_10.columns.tolist()

['states',
 'nic_name',
 'First_Tag',
 'Tidy_Desc',
 'Tidy_Desc_Stemmed',
 'Tidy_Desc_Lemma']

In [167]:
train_10_ana = train_10[['Tidy_Desc_Lemma','First_Tag']]

In [168]:
#pre-processing
import re 
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\n", "", string)    
    string = re.sub(r"\r", "", string) 
    string = re.sub(r"[0-9]", "digit", string)
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

In [169]:
#train test split
from sklearn.model_selection import train_test_split
X = []
for i in range(train_10_ana.shape[0]):
    X.append(clean_str(train_10_ana.iloc[i][0]))
y = np.array(train_10_ana["First_Tag"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5)

In [170]:
#feature engineering and model selection
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [172]:
#pipeline of feature engineering and model
model = Pipeline([('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

In [95]:
#paramater selection
from sklearn.model_selection import GridSearchCV
parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
               'tfidf__use_idf': (True, False)}

In [96]:
gs_clf_svm = GridSearchCV(model, parameters, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(X, y)
print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)

C:\Datascience\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Datascience\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Datascience\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Datascience\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Datascience\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `du

1.0
{'tfidf__use_idf': True, 'vectorizer__ngram_range': (1, 1)}


In [97]:
#preparing the final pipeline using the selected parameters
model = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

In [98]:
#fit model with training data
model.fit(X_train, y_train)

C:\Datascience\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Datascience\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Datascience\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Datascience\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Datascience\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `du

Pipeline(steps=[('vectorizer', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 OneVsRestClassifier(estimator=LinearSVC(class_weight='balanced')))])

In [99]:
#evaluation on test data
pred = model.predict(X_test)

In [100]:
model.classes_

array(['Crop and animal production, hunting and related service activities',
       'Education', 'Land transport and transport via pipelines',
       'Manufacture of food products',
       'Manufacture of other non-metallic mineral products',
       'Manufacture of textiles', 'Public administration and defence',
       'Retail trade, except of motor vehicles and motorcycles',
       'Specialized construction activities',
       'Wholesale trade, except of motor vehicles and motorcycles'],
      dtype='<U66')

In [101]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(pred, y_test)

array([[2210,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 1475,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,  989,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0, 1961,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,  934,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0, 1064,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0, 1093,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0, 3408,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0, 1109,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0, 1728]],
      dtype=int64)

In [102]:
accuracy_score(y_test, pred)

1.0

# Using k-fold cross-validation

In [103]:
from sklearn.model_selection import cross_val_score

In [174]:
import re

# Define a function to clean and preprocess the category names
def clean_category_name(category):
    # Convert to lowercase
    category = category.lower()
    # Remove any non-alphanumeric characters except spaces
    category = re.sub(r'[^a-zA-Z\s]', '', category)
    # Remove extra spaces
    category = category.strip()
    # Remove common words like 'of'
    stop_words = set(stopwords.words('english'))
    category = ' '.join([word for word in category.split() if word not in stop_words])
    return category

# Example usage:
categories = {
    'Construction of buildings': 1353,
    'Activities of households as employers of domestic personnel': 1218,
    'Manufacture of furniture': 1146,
    'Water collection, treatment and supply': 1104,
    'Veterinary activities': 1029
}

# Clean and preprocess category names
cleaned_categories = {clean_category_name(category): count for category, count in categories.items()}

print(cleaned_categories)


{'construction buildings': 1353, 'activities households employers domestic personnel': 1218, 'manufacture furniture': 1146, 'water collection treatment supply': 1104, 'veterinary activities': 1029}


In [176]:
# 1. Split Data into Train and Test Sets (already done)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5)

# 2. Define the Model (using the existing pipeline)
model = Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))
])

In [177]:
# 3. Cross-Validation
# Perform 5-fold cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

C:\Datascience\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Datascience\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Datascience\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Datascience\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Datascience\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `du

In [178]:
# 4. Average Accuracy
average_accuracy = cv_scores.mean()

In [179]:
# Print the average accuracy
print("Average Accuracy:", average_accuracy)

Average Accuracy: 1.0


# Checking using Random Forest Classifier

In [180]:
from sklearn.ensemble import RandomForestClassifier

import re

# Define a function to clean and preprocess the category names
def clean_category_name(category):
    # Convert to lowercase
    category = category.lower()
    # Remove any non-alphanumeric characters except spaces
    category = re.sub(r'[^a-zA-Z\s]', '', category)
    # Remove extra spaces
    category = category.strip()
    # Remove common words like 'of'
    stop_words = set(stopwords.words('english'))
    category = ' '.join([word for word in category.split() if word not in stop_words])
    return category

# Example usage:
categories = {
    'Construction of buildings': 1353,
    'Activities of households as employers of domestic personnel': 1218,
    'Manufacture of furniture': 1146,
    'Water collection, treatment and supply': 1104,
    'Veterinary activities': 1029
}

# Clean and preprocess category names
cleaned_categories = {clean_category_name(category): count for category, count in categories.items()}

# Define class weights based on the class distribution
class_weights = {'Manufacture of furniture': 1,
                 'Water collection, treatment and supply': 1,
                 'Construction of buildings': 1,
                 'Veterinary activities': 1,
                 'Primary education': 1,
                 'Other human health activities': 1,
                 'Activities of households as employers of domestic personnel': 1,
                 'Blank': 0.5,  # Adjust the weight for the "Blank" class
                 'Incomplete description/ Wrongly Classifed': 0.5,  # Adjust the weight for the "Incomplete description/Wrongly Classified" class
                 'Manufacture of tobacco products': 1}

# Define the pipeline with RandomForestClassifier
model_rf = Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weights))
])

# Fit the model with training data
model_rf.fit(X_train, y_train)

# Evaluate on test data
pred_rf = model_rf.predict(X_test)

# Calculate accuracy
accuracy_rf = accuracy_score(y_test, pred_rf)
print("Accuracy with Random Forest Classifier after adjusting for class imbalance:", accuracy_rf)


Accuracy with Random Forest Classifier after adjusting for class imbalance: 1.0
